In [18]:
import numpy as np
import pandas as pd
import glob
import os
import cv2
from keras.layers import Input
from sklearn.preprocessing import OneHotEncoder
from keras.applications import vgg16
from tensorflow.keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.models import Sequential,Model
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import RandomSearch
print("imported")

imported


In [2]:
df = pd.read_csv("C:\\MyLearnings\\DeepLearning\\Projects\\JanataHack\\train_SOaYf6m\\train.csv")
print(df.head(5))
print(df['emergency_or_not'].value_counts(ascending = True))
path = "C:\\MyLearnings\\DeepLearning\\Projects\\JanataHack\\train_SOaYf6m\\images\\"

  image_names  emergency_or_not
0    1503.jpg                 0
1    1420.jpg                 0
2    1764.jpg                 0
3    1356.jpg                 0
4    1117.jpg                 0
1    681
0    965
Name: emergency_or_not, dtype: int64


In [22]:
#train dataset
labels = []
images= []
for itr in range(len(df)):
    image_path = os.path.join(path , df['image_names'][itr])
    img = cv2.imread(image_path)
    img = cv2.resize(img,(128,128))
    images.append(img)
    labels.append(df['emergency_or_not'][itr])
images = np.array(images)
labels = np.array(labels)
print("done")

done


In [24]:
def build_model(hp):
    num_layers = hp.Int('num_layers', 2, 8, default=6)
    lr = hp.Choice('learning_rate', [1e-3, 5e-4])

    inputs = layers.Input(shape=(128, 128, 3))
    x = inputs

    for idx in range(num_layers):
        idx = str(idx)

        filters = hp.Int('filters_' + idx, 32, 256, step=32, default=64)
        x = layers.Conv2D(filters=filters, kernel_size=3, padding='same',
                          activation='relu')(x)

        # add a pooling layers if needed
        if x.shape[1] >= 8:
            pool_type = hp.Choice('pool_' + idx, values=['max', 'avg'])
            if pool_type == 'max':
                x = layers.MaxPooling2D(2)(x)
            elif pool_type == 'avg':
                x = layers.AveragePooling2D(2)(x)

    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    # Build model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=Adam(lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model
    

In [25]:
tuner = RandomSearch(build_model, objective='accuracy', max_trials=5)

# Display search space overview
tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


In [26]:
tuner.search(images, labels, batch_size=8, epochs=10)


Train on 1646 samples
Epoch 1/10
1472/1646 [=========================>....] - ETA: 47:27 - loss: 9.3149 - accuracy: 0.500 - ETA: 23:52 - loss: 96.1978 - accuracy: 0.56 - ETA: 15:59 - loss: 89.1679 - accuracy: 0.50 - ETA: 12:04 - loss: 70.9387 - accuracy: 0.40 - ETA: 9:42 - loss: 57.1467 - accuracy: 0.4000 - ETA: 8:07 - loss: 47.7865 - accuracy: 0.437 - ETA: 7:00 - loss: 41.1931 - accuracy: 0.428 - ETA: 6:09 - loss: 36.7254 - accuracy: 0.437 - ETA: 5:30 - loss: 32.9871 - accuracy: 0.416 - ETA: 4:58 - loss: 29.8059 - accuracy: 0.450 - ETA: 4:32 - loss: 27.2293 - accuracy: 0.477 - ETA: 4:10 - loss: 25.0005 - accuracy: 0.510 - ETA: 3:52 - loss: 23.1523 - accuracy: 0.519 - ETA: 3:36 - loss: 21.5694 - accuracy: 0.517 - ETA: 3:23 - loss: 20.1799 - accuracy: 0.516 - ETA: 3:11 - loss: 18.9603 - accuracy: 0.531 - ETA: 3:00 - loss: 17.9136 - accuracy: 0.522 - ETA: 2:51 - loss: 16.9664 - accuracy: 0.513 - ETA: 2:43 - loss: 16.1022 - accuracy: 0.526 - ETA: 2:35 - loss: 15.3309 - accuracy: 0.525 - E

Train on 1646 samples
Epoch 1/10
1504/1646 [==========================>...] - ETA: 7:33 - loss: 1.3903 - accuracy: 0.50 - ETA: 3:56 - loss: 4.7799 - accuracy: 0.68 - ETA: 2:42 - loss: 7.4303 - accuracy: 0.66 - ETA: 2:06 - loss: 6.2032 - accuracy: 0.65 - ETA: 1:44 - loss: 5.1032 - accuracy: 0.62 - ETA: 1:29 - loss: 4.4625 - accuracy: 0.60 - ETA: 1:19 - loss: 3.9664 - accuracy: 0.55 - ETA: 1:11 - loss: 3.5688 - accuracy: 0.51 - ETA: 1:05 - loss: 3.2481 - accuracy: 0.51 - ETA: 1:00 - loss: 2.9915 - accuracy: 0.52 - ETA: 56s - loss: 2.7863 - accuracy: 0.5227 - ETA: 52s - loss: 2.6104 - accuracy: 0.552 - ETA: 50s - loss: 2.4623 - accuracy: 0.557 - ETA: 47s - loss: 2.3358 - accuracy: 0.544 - ETA: 45s - loss: 2.2241 - accuracy: 0.550 - ETA: 43s - loss: 2.1271 - accuracy: 0.562 - ETA: 41s - loss: 2.0414 - accuracy: 0.558 - ETA: 40s - loss: 1.9672 - accuracy: 0.555 - ETA: 39s - loss: 1.9015 - accuracy: 0.552 - ETA: 37s - loss: 1.8385 - accuracy: 0.550 - ETA: 36s - loss: 1.7839 - accuracy: 0.553

Train on 1646 samples
Epoch 1/10
1504/1646 [==========================>...] - ETA: 6:40 - loss: 0.6663 - accuracy: 0.75 - ETA: 3:30 - loss: 15.7501 - accuracy: 0.562 - ETA: 2:27 - loss: 10.9679 - accuracy: 0.541 - ETA: 1:55 - loss: 8.4563 - accuracy: 0.562 - ETA: 1:36 - loss: 6.8492 - accuracy: 0.62 - ETA: 1:23 - loss: 5.9437 - accuracy: 0.60 - ETA: 1:14 - loss: 5.3382 - accuracy: 0.55 - ETA: 1:07 - loss: 4.8452 - accuracy: 0.51 - ETA: 1:01 - loss: 4.3658 - accuracy: 0.52 - ETA: 57s - loss: 4.0511 - accuracy: 0.5000 - ETA: 53s - loss: 3.7556 - accuracy: 0.477 - ETA: 51s - loss: 3.5084 - accuracy: 0.468 - ETA: 48s - loss: 3.3005 - accuracy: 0.461 - ETA: 46s - loss: 3.1111 - accuracy: 0.473 - ETA: 44s - loss: 2.9543 - accuracy: 0.458 - ETA: 42s - loss: 2.8123 - accuracy: 0.460 - ETA: 41s - loss: 2.6865 - accuracy: 0.470 - ETA: 39s - loss: 2.5753 - accuracy: 0.486 - ETA: 38s - loss: 2.4786 - accuracy: 0.486 - ETA: 37s - loss: 2.3888 - accuracy: 0.493 - ETA: 36s - loss: 2.3059 - accuracy: 

Train on 1646 samples
Epoch 1/10
1504/1646 [==========================>...] - ETA: 7:45 - loss: 1.2312 - accuracy: 0.37 - ETA: 4:01 - loss: 6.7260 - accuracy: 0.31 - ETA: 2:47 - loss: 4.6908 - accuracy: 0.37 - ETA: 2:09 - loss: 4.0299 - accuracy: 0.40 - ETA: 1:47 - loss: 3.3836 - accuracy: 0.40 - ETA: 1:32 - loss: 3.0932 - accuracy: 0.41 - ETA: 1:22 - loss: 2.7626 - accuracy: 0.41 - ETA: 1:13 - loss: 2.4825 - accuracy: 0.45 - ETA: 1:07 - loss: 2.3344 - accuracy: 0.44 - ETA: 1:02 - loss: 2.1815 - accuracy: 0.42 - ETA: 57s - loss: 2.0571 - accuracy: 0.4205 - ETA: 54s - loss: 1.9565 - accuracy: 0.416 - ETA: 51s - loss: 1.8679 - accuracy: 0.423 - ETA: 49s - loss: 1.7796 - accuracy: 0.446 - ETA: 46s - loss: 1.7103 - accuracy: 0.458 - ETA: 44s - loss: 1.6516 - accuracy: 0.453 - ETA: 43s - loss: 1.5920 - accuracy: 0.463 - ETA: 41s - loss: 1.5560 - accuracy: 0.458 - ETA: 40s - loss: 1.5078 - accuracy: 0.467 - ETA: 38s - loss: 1.4767 - accuracy: 0.450 - ETA: 37s - loss: 1.4421 - accuracy: 0.446

Train on 1646 samples
Epoch 1/10
1488/1646 [==========================>...] - ETA: 6:45 - loss: 4.6527 - accuracy: 0.75 - ETA: 3:37 - loss: 15.7316 - accuracy: 0.750 - ETA: 2:35 - loss: 12.5597 - accuracy: 0.666 - ETA: 2:04 - loss: 10.8546 - accuracy: 0.562 - ETA: 1:45 - loss: 10.8133 - accuracy: 0.525 - ETA: 1:32 - loss: 10.1798 - accuracy: 0.479 - ETA: 1:23 - loss: 9.0746 - accuracy: 0.500 - ETA: 1:17 - loss: 8.2666 - accuracy: 0.51 - ETA: 1:11 - loss: 7.8640 - accuracy: 0.50 - ETA: 1:07 - loss: 7.2389 - accuracy: 0.48 - ETA: 1:04 - loss: 6.6939 - accuracy: 0.50 - ETA: 1:01 - loss: 6.4698 - accuracy: 0.48 - ETA: 58s - loss: 6.0258 - accuracy: 0.5000 - ETA: 56s - loss: 5.6329 - accuracy: 0.508 - ETA: 54s - loss: 5.3332 - accuracy: 0.508 - ETA: 52s - loss: 5.0262 - accuracy: 0.531 - ETA: 51s - loss: 4.8212 - accuracy: 0.514 - ETA: 49s - loss: 4.6115 - accuracy: 0.513 - ETA: 48s - loss: 4.4033 - accuracy: 0.526 - ETA: 47s - loss: 4.2186 - accuracy: 0.537 - ETA: 46s - loss: 4.0665 - accu

INFO:tensorflow:Oracle triggered exit


In [27]:
tuner.results_summary()


In [28]:
# Retrieve the best model and display its architecture
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
average_pooling2d (AveragePo (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 192)       110784    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 192)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 256)       442624    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 256)       0     

In [30]:
scores = best_model.evaluate(images, labels, verbose=0)
print("%s: %.2f%%" % (best_model.metrics_names[1], scores[1]*100))


accuracy: 93.32%


In [40]:

inputs = layers.Input(shape=(128, 128, 3))
x = inputs


x = layers.Conv2D(64, kernel_size=3, padding='same',
activation='relu')(x)
x = layers.AveragePooling2D(2)(x)
x = layers.Conv2D(192, kernel_size=3, padding='same',
activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, kernel_size=3, padding='same',
activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(96, kernel_size=3, padding='same',
activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, kernel_size=3, padding='same',
activation='relu')(x)
x = layers.AveragePooling2D(2)(x)
x = layers.Conv2D(128, kernel_size=3, padding='same',
activation='relu')(x)
x = layers.Conv2D(64, kernel_size=3, padding='same',
activation='relu')(x)
x = layers.Conv2D(64, kernel_size=3, padding='same',
activation='relu')(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

# Build model
model = keras.Model(inputs, outputs)
model.compile(optimizer=Adam(0.0005),
loss='binary_crossentropy',
metrics=['accuracy'])
print(model.summary())

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 128, 128, 64)      1792      
_________________________________________________________________
average_pooling2d_10 (Averag (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 64, 64, 192)       110784    
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 32, 32, 192)       0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 32, 32, 256)       442624    
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 16, 16, 256)       0   

In [41]:
history = model.fit(images,labels, epochs =25)

Train on 1646 samples
Epoch 1/25
1646/1646 [==============================] - ETA: 50s - loss: 0.9353 - accuracy: 0.250 - ETA: 31s - loss: 3.6138 - accuracy: 0.421 - ETA: 25s - loss: 2.7002 - accuracy: 0.416 - ETA: 22s - loss: 2.3042 - accuracy: 0.453 - ETA: 20s - loss: 2.1623 - accuracy: 0.406 - ETA: 18s - loss: 1.9053 - accuracy: 0.453 - ETA: 17s - loss: 1.7455 - accuracy: 0.477 - ETA: 16s - loss: 1.6098 - accuracy: 0.492 - ETA: 16s - loss: 1.5077 - accuracy: 0.503 - ETA: 15s - loss: 1.4412 - accuracy: 0.496 - ETA: 14s - loss: 1.3691 - accuracy: 0.511 - ETA: 14s - loss: 1.3073 - accuracy: 0.520 - ETA: 13s - loss: 1.2535 - accuracy: 0.533 - ETA: 13s - loss: 1.2221 - accuracy: 0.542 - ETA: 12s - loss: 1.2044 - accuracy: 0.537 - ETA: 12s - loss: 1.1771 - accuracy: 0.535 - ETA: 11s - loss: 1.1455 - accuracy: 0.538 - ETA: 11s - loss: 1.1189 - accuracy: 0.539 - ETA: 10s - loss: 1.0912 - accuracy: 0.551 - ETA: 10s - loss: 1.0782 - accuracy: 0.540 - ETA: 10s - loss: 1.0583 - accuracy: 0.543 